In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
file_path = '/content/drive/MyDrive/Data Mining (CSE-4891)/Project files/dataset.csv'
import pandas as pd

df = pd.read_csv(file_path, delimiter=',', encoding='utf-8')

In [5]:
# Check unique classes and their counts and print the distribution
class_distribution = df['label'].value_counts()

print("Class Distribution:")
print(class_distribution)

Class Distribution:
label
Personal          4225
Geopolitical      3761
Religious         1714
Political         1596
Neutral            835
Gender abusive     316
Name: count, dtype: int64


In [6]:
df = df[df['label'] != 'Gender abusive']

# Check the new class distribution
class_distribution_after_drop = df['label'].value_counts()

# Print the updated distribution
print("Updated Class Distribution:")
print(class_distribution_after_drop)

Updated Class Distribution:
label
Personal        4225
Geopolitical    3761
Religious       1714
Political       1596
Neutral          835
Name: count, dtype: int64


In [1]:
!pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.6 MB/s eta 0:00:00


In [7]:
import re
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics import classification_report, accuracy_score
import datasets
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Clean text data
def clean_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove non-alphabetic characters
    text = text.lower()  # Convert to lowercase
    return text

df['banglish_clean'] = df['banglish'].apply(clean_text)

# Label Encoding labels
label_encoder = LabelEncoder()
df['label_encoded'] = label_encoder.fit_transform(df['label'])

# Splitting the data into train and test
X = df['banglish_clean']
y = df['label_encoded']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create Hugging Face Dataset
train_dataset = datasets.Dataset.from_dict({'text': X_train, 'label': y_train})
test_dataset = datasets.Dataset.from_dict({'text': X_test, 'label': y_test})

# Load Bangla BERT Tokenizer and Model
tokenizer = AutoTokenizer.from_pretrained('sagorsarker/bangla-bert-base')
bert_model = AutoModel.from_pretrained('sagorsarker/bangla-bert-base')

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Convert tokenized dataset to numpy arrays
X_train_tokens = np.array(train_dataset['input_ids'])
X_test_tokens = np.array(test_dataset['input_ids'])
y_train = np.array(y_train)
y_test = np.array(y_test)

# Build LSTM model
embedding_matrix = bert_model.embeddings.word_embeddings.weight.detach().numpy()
vocab_size, embedding_dim = embedding_matrix.shape

lstm_model = Sequential()
lstm_model.add(Input(shape=(128,)))
lstm_model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, weights=[embedding_matrix], input_length=128, trainable=False))
lstm_model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
lstm_model.add(Dense(64, activation='relu'))
lstm_model.add(Dropout(0.5))
lstm_model.add(Dense(len(label_encoder.classes_), activation='softmax'))

lstm_model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=2e-5), metrics=['accuracy'])

# Train LSTM model
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
lstm_model.fit(X_train_tokens, y_train, batch_size=32, epochs=10, validation_data=(X_test_tokens, y_test), callbacks=[early_stopping])

# Evaluate LSTM model
lstm_preds = np.argmax(lstm_model.predict(X_test_tokens), axis=-1)
lstm_accuracy = accuracy_score(y_test, lstm_preds)
print("\nBangla BERT + LSTM Model Accuracy:", lstm_accuracy)
print("\nBangla BERT + LSTM Classification Report:\n", classification_report(y_test, lstm_preds, target_names=label_encoder.classes_))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/491 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/2.24M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/660M [00:00<?, ?B/s]

Map:   0%|          | 0/9704 [00:00<?, ? examples/s]

Map:   0%|          | 0/2427 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
304/304 ━━━━━━━━━━━━━━━━━━━━ 118s 378ms/step - accuracy: 0.2939 - loss: 1.5677 - val_accuracy: 0.3494 - val_loss: 1.4587
Epoch 2/10
304/304 ━━━━━━━━━━━━━━━━━━━━ 120s 394ms/step - accuracy: 0.3255 - loss: 1.4883 - val_accuracy: 0.3494 - val_loss: 1.4566
Epoch 3/10
304/304 ━━━━━━━━━━━━━━━━━━━━ 135s 372ms/step - accuracy: 0.3379 - loss: 1.4817 - val_accuracy: 0.3494 - val_loss: 1.4553
Epoch 4/10
304/304 ━━━━━━━━━━━━━━━━━━━━ 141s 369ms/step - accuracy: 0.3383 - loss: 1.4802 - val_accuracy: 0.3494 - val_loss: 1.4556
Epoch 5/10
304/304 ━━━━━━━━━━━━━━━━━━━━ 143s 372ms/step - accuracy: 0.3400 - loss: 1.4752 - val_accuracy: 0.3494 - val_loss: 1.4549
Epoch 6/10
304/304 ━━━━━━━━━━━━━━━━━━━━ 151s 403ms/step - accuracy: 0.3314 - loss: 1.4807 - val_accuracy: 0.3494 - val_loss: 1.4520
Epoch 7/10
304/304 ━━━━━━━━━━━━━━━━━━━━ 135s 379ms/step - accuracy: 0.3339 - loss: 1.4807 - val_accuracy: 0.3494 - val_loss: 1.4530
Epoch 8/10
304/304 ━━━━━━━━━━━━━━━━━━━━ 115s 379ms/step - accuracy: 0.3445 -

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
